In [4]:
import json
from openai import OpenAI
from dotenv import dotenv_values
import minsearch

In [3]:
API_KEY = dotenv_values(".env")["API_KEY"]
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)
    
documents = []
for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [12]:
client = OpenAI(api_key=API_KEY, base_url="https://api.perplexity.ai")

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
    )
index.fit(documents)

In [24]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}
    results = index.search(
        query = query,
        filter_dict = {"course": "data-engineering-zoomcamp"}, # when we want to limit to data engineering zoomcamp course
        boost_dict = boost,
        num_results = 10
    )
    return results

In [25]:
def build_prompt(query, search_results): 
    propmt_template = """"You're a course teaching assistant. Answer the QUESTION based on CONTEXT. 
    Use only the facts from CONTEXT when answering the QUESTION.


    QUESTION: {question}

    CONTEXT: {context}""".strip() ## specify the role # prompt engineering 
    context = ""
    for doc in search_results:
        context = context + f"section : {doc['section']} \nquestion: {doc['question']} \nanswer: {doc['text']}\n\n"
    prompt = propmt_template.format(question = query, context = context).strip()
    return prompt

In [26]:
def llm(prompt):
    response = client.chat.completions.create(
    model = "llama-3-sonar-large-32k-chat",
    messages = [{"role": 'user', "content": prompt}],
    )
    return response.choices[0].message.content

In [32]:
def rag(query):
    #query = "how do i run kafka?"
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [33]:
rag("how do i run kafka ?")

'To run Kafka, you need to make sure that your Kafka broker Docker container is running. You can check this by running `docker ps` in your terminal. If the container is not running, navigate to the folder containing your Docker Compose YAML file and run `docker compose up -d` to start all the instances.'

In [ ]:
# REPLACING THE SEARCH WITH ELASTIC SEARCH:
